In [12]:
import ee
import xarray as xr
import numpy as np
import geopandas as gpd
import geemap

In [4]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [57]:
# Define the time range for the dataset extraction
start_period = '2014-01-01'
end_period = '2024-08-31'


In [ ]:
utm = "EPSG:32632"  # 'EPSG:4326' 'SR-ORG:6974' 'EPSG:23847'

In [58]:
terra = (
    ee.ImageCollection("MODIS/061/MOD13Q1")
    .select(["EVI", "SummaryQA", "DetailedQA"])
    .filterDate(start_period, end_period)
)
aqua = (
    ee.ImageCollection("MODIS/061/MYD13Q1")
    .select(["EVI", "SummaryQA", "DetailedQA"])
    .filterDate(start_period, end_period)
)

modis = terra.select("EVI").merge(aqua.select("EVI"))
modis = modis.sort("system:time_start")

In [59]:
# Function to extract bitwise quality flags
def bitwiseExtract(value, fromBit, toBit=None):
    if toBit is None:
        toBit = fromBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

# Function to apply MODIS QA mask
def modisQA_mask(image):
    sqa = image.select("SummaryQA")
    dqa = image.select("DetailedQA")
    viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
    viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
    viSnowIceFlagsD = bitwiseExtract(dqa, 14)
    mask = (
        viQualityFlagsS.eq(0)
        .And(viQualityFlagsD.eq(0))
        .And(viSnowIceFlagsD.eq(0))
    )
    return image.updateMask(mask)

In [60]:
# Apply the QA mask to Terra and Aqua datasets
mod13q1_QC = terra.map(modisQA_mask)
myd13q1_QC = aqua.map(modisQA_mask)

# Merge the cleaned Terra and Aqua datasets
mxd13q1_cleaned = mod13q1_QC.select("EVI").merge(myd13q1_QC.select("EVI"))
mxd13q1_cleaned_sorted = mxd13q1_cleaned.sort("system:time_start")
#modis_sorted = modis.sort("system:time_start")

# Integrate LGRIP crop data to filter out non-crop areas
lgrip30 = ee.ImageCollection("projects/sat-io/open-datasets/GFSAD/LGRIP30").mosaic()
crop_mask = lgrip30.select("b1").gt(1).rename("crop")

# Apply the crop mask to the MODIS data
def apply_crop_mask(image):
    return image.updateMask(crop_mask)

# Apply the crop mask to the cleaned MODIS data
mxd13q1_final = mxd13q1_cleaned_sorted.map(apply_crop_mask)
#modis_final = modis_sorted.map(apply_crop_mask)


In [ ]:
iso3 = "SYR"
country = "Syria"
release_type = "gbOpen"

adm = "ADM0"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm0_syr = gpd.read_file(res["gjDownloadURL"])

adm = "ADM1"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm1_syr = gpd.read_file(res["gjDownloadURL"])

adm = "ADM2"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm2_syr = gpd.read_file(res["gjDownloadURL"])


In [ ]:
#aoi = gpd_to_gee(adm0_ner)
aoi = gpd_to_gee(adm0_syr)

In [ ]:
ds_all = []
scale_factor = 0.0001
years = range(2001, 2024)
months = range(1, 13)
for year in tqdm(years):
    for month in tqdm(months):
        modis_sel = mxd13q1.filter(ee.Filter.calendarRange(year, year, "year")).filter(
            ee.Filter.calendarRange(month, month, "month")
        )
        # modis_sel = mxd13q1.filterDate(f'{year}-{month}-01', f'{year}-{month+1}-01')
        ds = xr.open_dataset(
            modis_sel,
            engine="ee",
            geometry=aoi.geometry(),
            crs=utm,
            scale=250,
        )
        ds = ds * scale_factor
        ds = ds.compute()
        ds_all.append(ds)
    # print(f"Finished {year}")

In [ ]:
ds = xr.concat(ds_all, dim="time")

In [39]:
'''def image_collection_to_xarray(collection, region, scale=1000):
    """Convert an Earth Engine ImageCollection to an Xarray dataset."""
    def reduce_image(image):
        values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=scale,
            maxPixels=1e9
        )
        return ee.Feature(None, values).set('system:time_start', image.get('system:time_start'))
    
    feature_collection = ee.FeatureCollection(collection.map(reduce_image))
    properties = feature_collection.aggregate_array('system:time_start').getInfo()
    data_values = feature_collection.aggregate_array('EVI').getInfo()
    
    dates = np.array(properties, dtype='datetime64[ms]')
    evi_data = np.array(data_values)
    
    ds = xr.Dataset(
        {"EVI": (["time"], evi_data)},
        coords={"time": dates},
        attrs={"description": "EVI data extracted from MODIS using Earth Engine"}
    )

    print(len(dates))
    print(len(evi_data))
    return ds'''

In [64]:
def image_collection_to_xarray(collection, region, scale=1000):
    """Convert an Earth Engine ImageCollection to an Xarray dataset."""
    
    def reduce_image(image):
        values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=scale,
            maxPixels=1e9
        )
        return ee.Feature(None, values).set('system:time_start', image.get('system:time_start'))
    
    # Map the reduce function over the ImageCollection to get the aggregated values
    feature_collection = ee.FeatureCollection(collection.map(reduce_image))
    
    # Extract the dates and EVI values from the FeatureCollection
    properties = feature_collection.aggregate_array('system:time_start').getInfo()
    data_values = feature_collection.aggregate_array('EVI').getInfo()

    # Filter out None values to ensure matching lengths
    valid_data = [(date, value) for date, value in zip(properties, data_values) if value is not None]

    # Check if we have valid data
    if len(valid_data) == 0:
        raise ValueError("No valid EVI data found for the specified region and time period.")

    # Unzip the list of tuples into separate lists for dates and EVI values
    dates, evi_data = zip(*valid_data)
    
    # Convert the dates into a numpy array of datetime objects
    dates = np.array(dates, dtype='datetime64[ms]')

    # Convert the EVI values into a numpy array
    evi_data = np.array(evi_data)

    # Create the Xarray dataset
    ds = xr.Dataset(
        {"EVI": (["time"], evi_data)},
        coords={"time": dates},
        attrs={"description": "EVI data extracted from MODIS using Earth Engine"}
    )

    return ds


In [61]:
#Load the Admin0 GeoJSON file into a GeoPandas DataFrame
geojson_path = "C:\\Users\\ishaa\\OneDrive\\Desktop\\DataLab\\Syria\\syria-economic-monitor\\data\\boundaries\\geoBoundaries-SYR-ADM0.geojson"
admin0_gdf = gpd.read_file(geojson_path)

# Convert the GeoPandas DataFrame to a GEE Geometry
region = geemap.geopandas_to_ee(admin0_gdf)

In [65]:
# Convert the ImageCollection to an Xarray dataset
ds = image_collection_to_xarray(modis, region, scale=500)


C:\Users\ishaa\AppData\Local\Temp\ipykernel_42792\1145512980.py:37: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds = xr.Dataset(


In [66]:
print(ds)

<xarray.Dataset> Size: 4kB
Dimensions:  (time: 260)
Coordinates:
  * time     (time) datetime64[ns] 2kB 2019-01-01 2019-01-09 ... 2024-08-20
Data variables:
    EVI      (time) float64 2kB 1.715e+03 1.752e+03 ... 1.083e+03 1.086e+03
Attributes:
    description:  EVI data extracted from MODIS using Earth Engine


In [67]:
import zarr
# Save the dataset to a Zarr format
zarr_output_path = "modis_evi_2014_2024_2.zarr"
ds.to_zarr(zarr_output_path, mode='w')
print(f"Data saved to Zarr format at {zarr_output_path}")

Data saved to Zarr format at modis_evi_2014_2024_2.zarr
